# Build (Extract, Transform, and Load)
Builds the `etl` table from the `ee`, `gegd`, and `mgp` tables.

### Import libraries

In [1]:
# Basic stack
from datetime import datetime

# Web Stack
import json
import requests

# Database stack
import sqlite3

# Data Science stack
import shapely.wkt
import pandas as pd
import geopandas as gpd
import folium

### User defined variables

In [2]:
db = "../../db.sqlite3"

### Drop table
This is for demonstration purposes

In [3]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS etl''')
conn.commit()
conn.close()

### Create `etl` table from selection of `ee`, `gegd`, and `mgp`
Might want to include the following other fields: cloud cover, satellite, sensor, number of bands, among others.

In [4]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE etl AS    
        SELECT 'EE' AS table_name,
               e.aoi_id_id AS aoi_id,
               e.catalog_id AS id,
               e.vendor_id AS vendor_id,
               e.entity_id AS entity_id,
               e.vendor as vendor,
               e.satellite AS platform,
               e.pixel_size_x AS pixel_size_x,
               e.pixel_size_y AS pixel_size_y,
               e.acquisition_date AS date,
               Date(e.publish_date) AS publish_date,
               AsText(e.bounds) as geometry
           FROM whale_earthexplorer e
        
        UNION
        
        SELECT 'GEGD' AS table_name,
               g.aoi_id_id AS aoi_id,
               g.legacy_id AS id,
               NULL AS vendor_id,
               NULL AS entity_id,
               g.company_name as vendor,
               g.source AS platform,
               g.per_pixel_x AS pixel_size_x,
               g.per_pixel_y AS pixel_size_y,
               Date(g.acquisition_date) AS date,
               NULL AS publish_date,
               AsText(g.geometry) AS geometry
           FROM whale_geointdiscovery g
        LEFT JOIN whale_earthexplorer e ON g.legacy_id = e.catalog_id
            WHERE
                e.catalog_id IS NULL
        
        UNION

        SELECT 'MGP' AS table_name,
               m.aoi_id_id AS aoi_id,
               m.id AS id,
               NULL AS vendor_id,
               NULL AS entity_id,
               "Maxar" AS vendor,
               m.platform AS platform,
               m.gsd AS pixel_size_x,
               m.gsd AS pixel_size_y,
               Date(datetime) AS date,
               NULL AS publish_date,
               AsText(m.bbox) AS geometry
           FROM whale_maxargeospatialplatform m
        LEFT JOIN whale_earthexplorer e ON m.id = e.catalog_id
        LEFT JOIN whale_geointdiscovery g ON m.id = g.legacy_id
            WHERE
                e.catalog_id IS NULL AND g.legacy_id IS NULL;
''')

c.execute('''ALTER TABLE etl ADD COLUMN sea_state_qual VARVHAR(15)''')
c.execute('''ALTER TABLE etl ADD COLUMN sea_state_quant NUMERIC(2, 2)''')
c.execute('''ALTER TABLE etl ADD COLUMN shareable VARVHAR(3)''')

conn.commit()
conn.close()

### Select new table and show it

In [5]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

df = pd.read_sql_query(f"SELECT * FROM etl", conn)
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

print("Your database shape is: {}\n".format(df.shape))
df_ee = df[df['table_name'] == 'EE']
print("Your database has {} records from EarthExplorer!".format(len(df_ee)))
print("\tYour EarthExplorer records have {} unique Catalog IDs!".format(len(df_ee['id'].unique())))
print("\tYour EarthExplorer records have {} unique Entity IDs!".format(len(df_ee['entity_id'].unique())))
print("Your database has {} records from GEGD!".format(len(df[df['table_name'] == 'GEGD'])))
print("Your database has {} records from MGP!\n".format(len(df[df['table_name'] == 'MGP'])))

df.head()

Your database shape is: (1497, 15)

Your database has 1434 records from EarthExplorer!
	Your EarthExplorer records have 132 unique Catalog IDs!
	Your EarthExplorer records have 1434 unique Entity IDs!
Your database has 3 records from GEGD!
Your database has 60 records from MGP!



,table_name,aoi_id,id,vendor_id,entity_id,vendor,platform,pixel_size_x,pixel_size_y,date,publish_date,geometry,sea_state_qual,sea_state_quant,shareable
0,EE,1,1040010096C03300,24JUL04205746-M1BS-508530682010_02_P001,WV320240704205746M00,MAXAR TECHNOLOGIES,WORLDVIEW-3,1.4,1.4,2024-07-04,2024-07-05,"POLYGON ((-151.53999 60.228081, -151.53999 60....",None,None,None
1,EE,1,1040010096C03300,24JUL04205746-P1BS-508530682010_02_P001,WV320240704205746P00,MAXAR TECHNOLOGIES,WORLDVIEW-3,0.3,0.3,2024-07-04,2024-07-05,"POLYGON ((-151.539477 60.22815, -151.539477 60...",None,None,None
2,EE,1,1040010096C03300,24JUL04205747-M1BS-508530682010_02_P002,WV320240704205747M00,MAXAR TECHNOLOGIES,WORLDVIEW-3,1.4,1.4,2024-07-04,2024-07-05,"POLYGON ((-151.538816 60.298098, -151.538816 6...",None,None,None
3,EE,1,1040010096C03300,24JUL04205747-M1BS-508530682010_02_P003,WV320240704205747M01,MAXAR TECHNOLOGIES,WORLDVIEW-3,1.4,1.4,2024-07-04,2024-07-05,"POLYGON ((-151.537566 60.367169, -151.537566 6...",None,None,None
4,EE,1,1040010096C03300,24JUL04205747-P1BS-508530682010_02_P002,WV320240704205747P01,MAXAR TECHNOLOGIES,WORLDVIEW-3,0.3,0.3,2024-07-04,2024-07-05,"POLYGON ((-151.538306 60.298164, -151.538306 6...",None,None,None


### Plot Images on an Interactive Map
USGS EarthExplorer footprints will be blue. GEGD footprints will be teal. Maxar Geospatial Platform footprints will be mustard.

In [6]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf[gdf['table_name'] == 'EE']['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

folium.GeoJson(
    gdf[gdf['table_name'] == 'GEGD']['geometry'].to_json(),
    style_function = lambda x: style_function('#037c6e')
).add_to(map)

folium.GeoJson(
    gdf[gdf['table_name'] == 'MGP']['geometry'].to_json(),
    style_function = lambda x: style_function('#DAA520')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End